In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ROOT
import os
# ROOT.enableJSVis()
c1 = ROOT.TCanvas()
ROOT.gSystem.Load("/usr/local/software/quadis/build/lib/libquadis.so")   

Welcome to JupyROOT 6.22/02


0

In [2]:
nSamples = 4096#Lets keep the first 200 entries = 0
SP = 8#1ns,check in SigGen

outfile = ROOT.TFile("/home/vasu/PPC/PPC_SigGen/zscan_1000/data36.root", "recreate")
tree = ROOT.TTree("T1","Data Pulses")
event = ROOT.NEvent()
tree.Branch("NEvent",event,32000,1);

pulse = ROOT.NPulse(nSamples)
pulse.SetEncodingScheme(1) #kDeltaVarInt?
pulse.SetSamplingPeriod(SP)

pulseData = ROOT.NPulseData()
pulseData.SetDigitizerType(2) #s3316
pulseData.AddParameterUL("TimeStamp")
pulseData.AddParameterUL("MaxEnergy")
pulseData.AddParameterUL("PeakingVal")
pulseData.AddParameterUL("StartEnergy")
pulseData.AddParameterUL("Channel")
pulseData.AddParameterUL("Pileup")
pulseData.AddParameterUL("UnixStartTime")
pulseData.AddParameterUL("Downsample")


infile1 = ROOT.TFile("/data/ppc/ortecppc/T1/SCAN100016/SCAN100016_36.root")
tree1 = infile1.Get("T1")
nevent1 = ROOT.NEvent()
tree1.SetBranchAddress("NEvent",nevent1)
nf = tree1.GetEntries()

blrPulse = ROOT.NPulse()
blrFilter = ROOT.NFBaselineRemover(1000) 
trapPulse = ROOT.NPulse()

count = 0

for i in range (nf):
    tree1.GetEntry(i)
    npulse = nevent1.GetPulse(0)
    TS = nevent1.GetPulseData(0).GetParameterByNameUL("TimeStamp")
    ME = nevent1.GetPulseData(0).GetParameterByNameUL("MaxEnergy")
    PV = nevent1.GetPulseData(0).GetParameterByNameUL("PeakingVal")
    SE = nevent1.GetPulseData(0).GetParameterByNameUL("StartEnergy")
    Ch = nevent1.GetPulseData(0).GetParameterByNameUL("Channel")
    PU = nevent1.GetPulseData(0).GetParameterByNameUL("Pileup")
    UST = nevent1.GetPulseData(0).GetParameterByNameUL("UnixStartTime")
    DS = nevent1.GetPulseData(0).GetParameterByNameUL("Downsample")
   
    blrFilter.Filter(npulse,blrPulse)
    baseline = blrFilter.GetCalcParameterByName("Baseline")
    trapFilter = ROOT.NFTrapezoid(750,250,0,baseline)
    trapFilter.Filter(blrPulse,trapPulse)
    val = trapFilter.GetCalcParameterByName("Max")
    if (count>=1000):
        break
    else:
        if (72.5<=val<=74.5):    
            count = count+1
            for j in range (nSamples):
                pulse[j] = blrPulse[j]
            pulseData.SetParameterByNameUL("TimeStamp",TS)
            pulseData.SetParameterByNameUL("MaxEnergy",ME)
            pulseData.SetParameterByNameUL("PeakingVal",PV)
            pulseData.SetParameterByNameUL("StartEnergy",SE)
            pulseData.SetParameterByNameUL("Channel",Ch)
            pulseData.SetParameterByNameUL("Pileup",PU)
            pulseData.SetParameterByNameUL("UnixStartTime",UST)
            pulseData.SetParameterByNameUL("Downsample",DS)
            event.AddPulseAndData(pulse,pulseData,0)

            tree.Fill()
            event.Clear()

outfile.cd()
tree.Write()
outfile.Close()

In [3]:
infile1 = ROOT.TFile("/home/vasu/PPC/PPC_SigGen/zscan_1000/data36.root")
tree1 = infile1.Get("T1")
event1 = ROOT.NEvent()
tree1.SetBranchAddress("NEvent",event1)
print (tree1.GetEntries())
print (count)

1000
1000
